In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

## New York City Green Taxi in the First Quarter of 2019.

In [3]:
# 2019 Yellow Taxi Trip Data
def extract_data(url, query):
    new_url = url + query
    df = pd.DataFrame()
    
    i = 0
    cur_page = new_url + "&$limit=1000&$offset=" + str(i)
    cur_data = pd.read_json(cur_page)
    while len(cur_data) >= 1000:
        df = df.append(cur_data, sort=True)
        i += 1000
        cur_page = new_url + "&$limit=1000&$offset=" + str(i)
        cur_data = pd.read_json(cur_page)
        
        if i % 4e5 == 0:
            print("{} records have been extracted at {}.".format(i, datetime.now().time()))
    
    if len(cur_data) > 0:
        df = df.append(cur_data, sort=True)
    
    return df

In [4]:
def save_to_csv(df, fileName="./data/myData.csv"):
    df.to_csv(fileName)

In [5]:
def get_data():
    url = "https://data.cityofnewyork.us/resource/2upf-qytp.json?"
    # pickup time: Jan. ~ June, pickup location: <= 50
    query = "$where=tpep_pickup_datetime between '2019-01-01' and '2019-04-01' and PULocationID<=50"
    query = query.replace(" ", "%20")
    print(url + query)  # first page

    print("Start time: ", datetime.now().time())
    df = extract_data(url, query)
    print("End time: ", datetime.now().time())

    save_to_csv(df)

In [6]:
def read_from_csv(path):
    df = pd.read_csv(path, index_col=0)
    return df

## Data Description and Exploration

| | **Taxi Information** | 
|----------|:-------------|
| VendorID | A code indicating the LPEP provider that provided the record. (1= Creative Mobile Technologies, LLC 2= VeriFone Inc.)|
| tpep_pickup_datetime | The date and time when the meter was engaged.|
| tpep_dropoff_datetime | The date and time when the meter was disengaged.|
| Passenger_count| The number of passengers in the vehicle.|
| Trip_distance | The elapsed trip distance in miles reported by the taximeter.|
| PULocationID | TLC Taxi Zone in which the taximeter was engaged|
| DOLocationID | TLC Taxi Zone in which the taximeter was disengaged|
| RateCodeID | The final rate code in effect at the end of the trip.(1= Standard rate 2=JFK 3=Newark 4=Nassau or Westchester 5=Negotiated fare 6=Group ride)|  
| Payment_type| A numeric code signifying how the passenger paid for the trip.(1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided trip)|
| Fare_amount | The time-and-distance fare calculated by the meter.|
|Extra | Miscellaneous extras and surcharges. Currently, this only includes the $0.50 and $1 rush hour and overnight charges.|
| MTA_tax | \$0.50 MTA tax that is automatically triggered based on the metered rate in use.|
| Improvement_surcharge | \$0.30 improvement surcharge assessed on hailed trips at the flag drop.|
| Tip_amount Tip amount | This field is automatically populated for credit card tips. Cash tips are not included.|
| Tolls_amount | Total amount of all tolls paid in trip.|
| Total_amount | The total amount charged to passengers. Does not include cash tips.|
| Trip_type | A code indicating whether the trip was a street-hail or a dispatch (1= Street-hail 2= Dispatch)|




### Data Clean

In [15]:
def drop_useless_columns(df):
    # get rid of useless columns
    df = df.drop(columns=["congestion_surcharge", "store_and_fwd_flag"])
    return df

In [16]:
def convert_to_timestamp(df):
    df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
    df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
    return df

In [17]:
def create_duration_colum(df):
    # create "duration" column from pickup and dropoff datetime
    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    return df

In [18]:
path = "./data/myData.csv"
df = read_from_csv(path)
df = drop_useless_columns(df)
df = convert_to_timestamp(df)
df = create_duration_colum(df)
df.head()

,dolocationid,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,payment_type,pulocationid,ratecodeid,tip_amount,tolls_amount,total_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,vendorid,duration
0,1,0.0,20.0,0.3,0.5,1,1,1,5,1.00,0.0,21.80,2019-01-01 02:19:32,2019-01-01 02:19:27,0.0,2,00:00:05
1,1,0.0,85.0,0.3,0.0,1,1,1,5,2.00,0.0,87.30,2019-01-01 05:52:53,2019-01-01 05:52:45,0.0,2,00:00:08
2,1,0.0,80.0,0.3,0.0,1,1,1,5,0.00,0.0,80.30,2019-01-01 06:23:23,2019-01-01 06:22:24,0.0,2,00:00:59
3,1,0.0,80.0,0.3,0.0,2,1,1,5,16.06,0.0,96.36,2019-01-01 08:13:01,2019-01-01 08:12:51,0.0,2,00:00:10
4,1,0.0,160.0,0.3,0.5,2,1,1,5,0.00,0.0,160.80,2019-01-01 08:14:53,2019-01-01 08:14:48,0.0,2,00:00:05
